<a href="https://colab.research.google.com/github/IvanKatorgin/NLP/blob/main/%D0%9E%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_%D0%B5%D1%81%D1%82%D0%B5%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D0%BE%D0%B3%D0%BE_%D1%8F%D0%B7%D1%8B%D0%BA%D0%B0_%D0%B4%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Курс "Обработка естественного языка". Домашнее задание 2. Классификация в АОТ. Каторгин И.П.

## Задание
Сделать классификацию данных fakenews

Используя ноутбук занятия (также размещен в папке Materials) и данные fakenews, 3 раза разными способами получить на задаче классификации значение f1 выше 0.91 для методов на sklearn и выше 0.52 для методов на pytorch.

Загрузим и посмотрим на данные

In [ ]:
!wget https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv

--2025-03-31 15:52:06--  https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1253562 (1.2M) [text/plain]
Saving to: ‘Constraint_Train.csv.1’

Constraint_Train.cs 100%[===================>]   1.20M  --.-KB/s    in 0.006s  

2025-03-31 15:52:07 (207 MB/s) - ‘Constraint_Train.csv.1’ saved [1253562/1253562]



In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('Constraint_Train.csv')

In [ ]:
df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [ ]:
from nltk.tokenize import word_tokenize
from tqdm import tqdm

In [ ]:
import nltk
nltk.data.path.append("/path/to/nltk_data")  # Replace with your desired path

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_r

True

In [ ]:
sentences = [word_tokenize(text.lower()) for text in tqdm(df.tweet)]

100%|██████████| 6420/6420 [00:01<00:00, 4878.50it/s]


In [ ]:
!pip install numpy==1.23.5
!pip install gensim

In [ ]:
from gensim import corpora, models

In [ ]:
from gensim.models.word2vec import Word2Vec
%time model_tweets = Word2Vec(sentences, workers=4, vector_size=300, min_count=3, window=5, sg=15)

CPU times: user 8.94 s, sys: 44.6 ms, total: 8.98 s
Wall time: 5.11 s


In [ ]:
model_tweets.wv.most_similar('vaccine')

[('developed', 0.8649674654006958),
 ('drug', 0.855300784111023),
 ('wave', 0.8533533811569214),
 ('trial', 0.8437551259994507),
 ('combat', 0.8341242671012878),
 ('against', 0.8321012258529663),
 ('scientists', 0.8306623101234436),
 ('cure', 0.8242757320404053),
 ('ready', 0.8212168216705322),
 ('hydroxychloroquine', 0.8204939365386963)]

In [ ]:
model_tweets.init_sims()

<ipython-input-22-f1a8336e4a1e>:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model_tweets.init_sims()


In [ ]:
import numpy as np

In [ ]:
def get_text_embedding(text):
    result = []
    for word in word_tokenize(text.lower()):
        if word in model_tweets.wv:
            result.append(model_tweets.wv[word])

    if len(result):
        result = np.average(result, axis=0)
    else:
        result = np.zeros(300)
    return result

In [ ]:
features = [get_text_embedding(text) for text in tqdm(df.tweet)]


100%|██████████| 6420/6420 [00:02<00:00, 2612.24it/s]


Построим модель логистической регрессии

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, df.label, test_size=0.33)

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
predicted = model.predict(X_test)

In [ ]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.92      0.89      0.90       997
        real       0.90      0.93      0.92      1122

    accuracy                           0.91      2119
   macro avg       0.91      0.91      0.91      2119
weighted avg       0.91      0.91      0.91      2119



Используем самый наивный метод

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vec = CountVectorizer()

In [ ]:
bow = vec.fit_transform(df.tweet)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(bow, df.label, test_size=0.33)
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.92      0.92      0.92      1038
        real       0.93      0.93      0.93      1081

    accuracy                           0.92      2119
   macro avg       0.92      0.92      0.92      2119
weighted avg       0.92      0.92      0.92      2119



Конечно, мы всегда можем поиграться с предобработкой.

In [ ]:
df.tweet

,tweet
0,The CDC currently reports 99031 deaths. In gen...
1,States reported 1121 deaths a small rise from ...
2,Politically Correct Woman (Almost) Uses Pandem...
3,#IndiaFightsCorona: We have 1524 #COVID testin...
4,Populous states can generate large case counts...
...,...
6415,A tiger tested positive for COVID-19 please st...
6416,???Autopsies prove that COVID-19 is??� a blood...
6417,_A post claims a COVID-19 vaccine has already ...
6418,Aamir Khan Donate 250 Cr. In PM Relief Cares Fund


Постоим модель PyTorch + LSTM

In [ ]:
labels = (df.label == 'real').astype(int).to_list()

Нужно заранее задать размер для максимальной длины предложений

In [ ]:
token_lists = [word_tokenize(text.lower()) for text in df.tweet]
max_len = len(max(token_lists, key=len))

In [ ]:
max_len

1592

Это слишком много. Но какая длина обычно?

In [ ]:
from collections import Counter
fd = Counter([len(tokens) for tokens in token_lists])

In [ ]:
fd.most_common(10)

[(20, 178),
 (25, 174),
 (22, 170),
 (18, 170),
 (19, 168),
 (21, 168),
 (16, 163),
 (17, 162),
 (15, 160),
 (23, 156)]

Зададим максимум 200

Возьмём те же w2v эмбеддинги

In [ ]:
def get_word_embedding(tokens, max_len):
    result = []
    for i in range(max_len):
        if i < len(tokens):
            word = tokens[i]
            if word in model_tweets.wv:
                result.append(model_tweets.wv[word])
            else:
                result.append(np.zeros(300))
        else:
            result.append(np.zeros(300))
    return result

In [ ]:
features = [get_word_embedding(text, 100) for text in tqdm(token_lists)]


100%|██████████| 6420/6420 [00:01<00:00, 5215.87it/s]


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(300, 100)
        self.out = nn.Linear(100, 1)

    def forward(self, x):
        embeddings, (shortterm, longterm) = self.lstm(x.transpose(0, 1))
        prediction = torch.sigmoid(self.out(longterm))
        return prediction


net = Net()
net.cuda()
print(net)

Net(
  (lstm): LSTM(300, 100)
  (out): Linear(in_features=100, out_features=1, bias=True)
)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.25)

In [ ]:
in_data = torch.tensor(X_train).float()
targets = torch.tensor(y_train).float()

<ipython-input-50-02003c1debcb>:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  in_data = torch.tensor(X_train).float()


In [ ]:
in_data.shape

torch.Size([4815, 100, 300])

In [ ]:
optimizer = optim.Adam(net.parameters(), lr=0.01)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
def train_one_epoch(in_data, targets, batch_size=16):
    for i in tqdm(range(0, in_data.shape[0], batch_size)):
        batch_x = in_data[i:i + batch_size].cuda()
        batch_y = targets[i:i + batch_size].cuda()
        optimizer.zero_grad()
        output = net(batch_x)
        loss = criterion(output.reshape(-1), batch_y)
        loss.backward()
        optimizer.step()
    print(loss)

In [ ]:
net.train()

Net(
  (lstm): LSTM(300, 100)
  (out): Linear(in_features=100, out_features=1, bias=True)
)

In [ ]:
for i in range(40):
  train_one_epoch(in_data, targets)


100%|██████████| 301/301 [00:01<00:00, 193.44it/s]


tensor(0.6931, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)



100%|██████████| 301/301 [00:00<00:00, 344.20it/s]


tensor(0.6931, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)



100%|██████████| 301/301 [00:00<00:00, 350.01it/s]


...

tensor(0.6931, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)



100%|██████████| 301/301 [00:00<00:00, 367.80it/s]


tensor(0.6931, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)



100%|██████████| 301/301 [00:00<00:00, 371.36it/s]

tensor(0.6931, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


Посмотрим, что получилось

In [ ]:
in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(y_test).float()

In [ ]:
with torch.no_grad():
    output = net(in_data_test.cuda()).reshape(-1)

In [ ]:
max(torch.nn.functional.softmax(output))

<ipython-input-58-c91dd9243900>:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  max(torch.nn.functional.softmax(output))


tensor(0.0006, device='cuda:0')

In [ ]:
targets_test

tensor([1., 1., 0.,  ..., 1., 0., 1.])

In [ ]:
result = (output.cpu() > 0.5) == targets_test

In [ ]:
result.sum().item() / len(result)

0.45981308411214955

Такую модель надо учить дольше

In [ ]:
features = [get_text_embedding(text) for text in tqdm(df.tweet)]


100%|██████████| 6420/6420 [00:02<00:00, 2693.98it/s]


In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.out = nn.Linear(300, 1)

    def forward(self, x):
        return torch.sigmoid(self.out(x))


net = Net()
net.cuda()
print(net)

Net(
  (out): Linear(in_features=300, out_features=1, bias=True)
)


In [ ]:
optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.25)

In [ ]:
in_data = torch.tensor(X_train).float()
targets = torch.tensor(y_train).float()

In [ ]:
def train_one_epoch(in_data, targets, batch_size=16):
    for i in tqdm(range(0, in_data.shape[0], batch_size)):
        batch_x = in_data[i:i + batch_size].cuda()
        batch_y = targets[i:i + batch_size].cuda()
        optimizer.zero_grad()
        output = net(batch_x)
        loss = criterion(output.squeeze(), batch_y)
        loss.backward()
        optimizer.step()
    print(loss)

In [ ]:
for i in range(20):
  train_one_epoch(in_data, targets)


100%|██████████| 301/301 [00:00<00:00, 961.67it/s] 


tensor(0.7634, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)



100%|██████████| 301/301 [00:00<00:00, 1096.97it/s]


tensor(0.7535, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)



100%|██████████| 301/301 [00:00<00:00, 1106.63it/s]


...

tensor(0.7134, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)



100%|██████████| 301/301 [00:00<00:00, 1077.26it/s]


tensor(0.7124, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)



100%|██████████| 301/301 [00:00<00:00, 1119.01it/s]

tensor(0.7115, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


In [ ]:
in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(y_test).float()

In [ ]:
with torch.no_grad():
    output = net(in_data_test.cuda()).squeeze(1)

In [ ]:
result = (output.cpu() > 0.5) == targets_test

In [ ]:
result.sum().item() / len(result)

0.621183800623053

Выводы:
- в работе 3 раза выполнена задача классификации данных fakenews разными способами используя методы sklearn и pytorch;
- значение f1 для методов sklearn составило 0.91 (логистическая регрессия) и 0,92 (самый наивный метод);
- значение f1 для метода pytorch (pytorch + LSTM) составило 0.62.